In [21]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.spatial.distance import cdist
from itertools import combinations

In [22]:
# Define the name for the output files and specify the source and destination paths
name = 'swc_skeleton'
path_source = '1. raw_data/'
path_save = '2. points_segments/'

# Read the SWC file into a DataFrame. SWC files are space-delimited.
df = pd.read_csv(path_source + '04546925828.swc', delimiter=' ')

# Extract segment information from the DataFrame
# The 'Parent' column specifies the ID of the parent point for a segment, and 'PointNo' is the ID of the current point.
segments = df[['Parent', 'PointNo']]
# Rename the columns for clarity
segments = segments.rename(columns={'Parent': 'pt_id1', 'PointNo': 'pt_id2'})
# Remove rows where either 'pt_id1' or 'pt_id2' is -1, indicating no connection or an invalid point
segments = segments[(segments['pt_id1'] != -1) & (segments['pt_id2'] != -1)]
# Reset the index of the DataFrame, dropping the old index
segments = segments.reset_index(drop=True)
# Rename the index column as 'seg_id' to uniquely identify each segment
segments = segments.rename(columns={'level_0': 'seg_id'})
# Save the segments information to a CSV file
segments.to_csv(path_save + name + '.segments.csv')

# Extract point information from the DataFrame
# Includes point ID, coordinates (X, Y, Z), and Radius
points = df[['PointNo', 'X', 'Y', 'Z', 'Radius']].copy()
# Rename columns for clarity and consistency
points = points.rename(columns={'PointNo': 'pt_id', 'X': 'x', 'Y': 'y', 'Z': 'z', 'Radius': 'radius'})
# Calculate the diameter of each point (neuron part) as twice the radius
points['diameter'] = points['radius'] * 2
# Reset the index of the DataFrame, ensuring a clean, sequential index
points = points.reset_index(drop=True)
# Save the points information to a CSV file
points.to_csv(path_save + name + '.points.csv')

In [23]:
points.head()

,pt_id,x,y,z,radius,diameter
0,0,2.661384e+06,476568.031250,924.013855,0.000000,0.000000
1,1,2.593444e+06,629145.228448,115464.111261,175.864052,351.728104
2,2,2.592194e+06,452332.997159,1527.023049,194.561144,389.122288
3,3,2.591064e+06,389871.937500,67386.000000,32.984845,65.969690
4,4,2.591080e+06,513114.760417,25360.628906,7.399605,14.799209


In [25]:
segments.head()

,pt_id1,pt_id2
0,66,382
1,386,383
2,383,384
3,380,385
4,387,386


In [27]:
def heal_skeletons(segments, points):
    # Initialize the graph
    G = nx.Graph()
    G.add_edges_from(segments[['pt_id1', 'pt_id2']].values)

    # Keep healing until all components are connected
    while True:
        components = list(nx.connected_components(G))
        if len(components) == 1:
            print("Skeleton is already fully connected.")
            break  # Exit the loop if only one component exists

        # Preparing for healing
        node_attributes = points.set_index('pt_id')[['x', 'y', 'z', 'radius']].to_dict('index')
        
        # Assume we haven't healed anything yet
        healed = False

        for component1, component2 in combinations(components, 2):
            component1_coords = np.array([[node_attributes[node]['x'], node_attributes[node]['y'], node_attributes[node]['z']] for node in component1])
            component2_coords = np.array([[node_attributes[node]['x'], node_attributes[node]['y'], node_attributes[node]['z']] for node in component2])
            distances = cdist(component1_coords, component2_coords, 'euclidean')

            min_distance_index = np.unravel_index(distances.argmin(), distances.shape)
            closest_nodes = list(component1)[min_distance_index[0]], list(component2)[min_distance_index[1]]

            # Creating a new segment between the closest nodes to heal the skeleton
            new_row = pd.DataFrame({'pt_id1': [closest_nodes[0]], 'pt_id2': [closest_nodes[1]]})
            segments = pd.concat([segments, new_row], ignore_index=True)
            
            # Update the graph with the new segment
            G.add_edge(closest_nodes[0], closest_nodes[1])
            
            healed = True  # We've made at least one healing connection
            break  # Break after the first healing to re-evaluate the components

        if not healed:
            print("No more healing possible.")
            break

    print("Healing complete. Number of disconnected components after healing:", len(list(nx.connected_components(G))))
    return segments, points

# Applying the healing function to data
path_save_2 = '3. healed_skeletons/'
segments_healed, points_healed = heal_skeletons(segments, points)
segments_healed.to_csv(path_save_2 + name + '.segments.csv')
points_healed.to_csv(path_save_2 + name+'.points.csv')

Skeleton is already fully connected.
Healing complete. Number of disconnected components after healing: 1
